In [1]:
import torch

from utils import *
%load_ext autoreload
%autoreload 2

In [2]:
dataset = dense_Dataset('Protein-Prediction-1/data/embeddings_subset.h5', 'Protein-Prediction-1/data/densities.csv', test_perc=0.15, val_perc=0.15) 

In [3]:
train_keys, val_keys = dataset.train_keys, dataset.val_keys
train_loader = CNN_Dataloader(dataset, train_keys[:len(train_keys) // 4], batch_size=64, shuffle=True)
val_loader = CNN_Dataloader(dataset, val_keys[:len(val_keys) // 4], batch_size=64, shuffle=True)

In [ ]:
hparams = {
    'epochs': 5,
    'lr':0.001,
    'device':'gpu'   
}

In [6]:
train_CNN(train_loader, val_loader, hparams)

starting cross validation
torch.Size([3, 1024, 413])
torch.Size([3, 512, 413])
torch.Size([3, 256, 413])
torch.Size([3, 413, 256])
torch.Size([3, 413, 1])
tensor([[0.0572, 0.0743, 0.0792,  ..., 0.0531, 0.0531, 0.0518],
        [0.0540, 0.0540, 0.0741,  ..., 0.0531, 0.0531, 0.0518],
        [0.0744, 0.0514, 0.0592,  ..., 0.0510, 0.0499, 0.0591]],
       grad_fn=<SqueezeBackward1>)
0.0019281703280285
torch.Size([3, 1024, 348])
torch.Size([3, 512, 348])
torch.Size([3, 256, 348])
torch.Size([3, 348, 256])
torch.Size([3, 348, 1])
tensor([[-0.2417, -0.3758, -0.3600,  ...,  0.0296,  0.0296,  0.0339],
        [-0.2105, -0.3444, -0.3249,  ...,  0.0296,  0.0296,  0.0339],
        [-0.1848, -0.3559, -0.3888,  ..., -0.5117, -0.4321, -0.2491]],
       grad_fn=<SqueezeBackward1>)
0.2452368038939312
torch.Size([3, 1024, 413])
torch.Size([3, 512, 413])
torch.Size([3, 256, 413])
torch.Size([3, 413, 256])
torch.Size([3, 413, 1])
tensor([[-0.1521, -0.2756, -0.2982,  ...,  0.0387,  0.0387,  0.0405],
     